In [1]:
import arcpy
import requests
import os
import zipfile
import io

#Assign working directory

In [2]:
workspace = r'C:\Users\cason\Desktop\GIS5571\Lab3\part1'

#Assign variables for data

In [3]:
mn_counties = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp'
mn_dem = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m'
mn_nlcd = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\NLCD_2019_Land_Cover.tif'
mn_streams = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\streams_with_strahler_stream_order.shp'
mn_roads = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\NLCD_2019_Land_Cover_Impervious_Descriptor.tif'

#Create study extent area from Minnesota Counties shapefile

In [4]:
# Select Wabasha, Winona, and Olmsted counties from mn_counties feature class
mn_county_selection = arcpy.conversion.FeatureClassToFeatureClass(mn_counties, 
                                                                  workspace, 
                                                                  "mn_counties_study_extent", 
                                                                  "CTY_NAME = 'Wabasha' Or CTY_NAME = 'Winona' Or CTY_NAME = 'Olmsted'", 
                                                                  r'AREA "AREA" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,AREA,-1,-1;PERIMETER "PERIMETER" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,PERIMETER,-1,-1;CTYONLY_ "CTYONLY_" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTYONLY_,-1,-1;CTYONLY_ID "CTYONLY_ID" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTYONLY_ID,-1,-1;COUN "COUN" true true false 4 Short 0 4,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,COUN,-1,-1;CTY_NAME "CTY_NAME" true true false 20 Text 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTY_NAME,0,20;CTY_ABBR "CTY_ABBR" true true false 4 Text 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTY_ABBR,0,4;CTY_FIPS "CTY_FIPS" true true false 4 Short 0 4,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTY_FIPS,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,Shape_Area,-1,-1', '')

In [5]:
# Dissolve selected counties to create study area extent boundary
arcpy.management.Dissolve(mn_county_selection, 
                          r'C:\Users\cason\Desktop\GIS5571\Lab3\part1\study_extent', 
                          None, 
                          None, 
                          "MULTI_PART", 
                          "DISSOLVE_LINES", 
                          '')

<Result 'C:\\Users\\cason\\Desktop\\GIS5571\\Lab3\\part1\\study_extent.shp'>

In [6]:
#Create variable for study extent area boundary
study_extent = r'C:\Users\cason\Desktop\GIS5571\Lab3\part1\study_extent'

#Roads: Extract raster by study extent mask and reclassify

In [7]:
#Use study extent as a mask for extracting roads raster
roads_study_extent = arcpy.sa.ExtractByMask(mn_roads, 
                                            study_extent, 
                                            "INSIDE", 
                                            '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); 
roads_study_extent.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\Lab3\Default.gdb\Extract_NLCD1")

In [9]:
#Reclassify masked roads raster. Unclassified: 10, Primary: 1, Secondary: 1, Tertiary: 1, Non-road/non-energy impervious: 2, LCMAP impervious: 3, Wind turbines: 7
roads_reclass = arcpy.ddd.Reclassify(roads_study_extent, 
                                     "Class_Name", 
                                     "Unclassified 10;'Primary road' 1;'Secondary road' 1;'Tertiary road' 1;'Non-road non-energy impervious' 2;'LCMAP impervious' 3;'Wind turbines' 7", 
                                     r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\roads_reclass", 
                                     "DATA")

#Slope: Extract raster by study extent, derive slope from DEM, and reclassify

In [10]:
#Use study extent as a mask for extracting DEM raster
dem_study_extent = arcpy.sa.ExtractByMask(mn_dem, 
                                          study_extent, 
                                          "INSIDE", 
                                          '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); 
dem_study_extent.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\dem_se")

In [11]:
#Use slope tool on masked DEM raster and use percent rise as output measurement
slope = arcpy.sa.Slope(dem_study_extent, 
                       "PERCENT_RISE", 
                       1, 
                       "PLANAR", 
                       "METER"); 
slope.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\slope")

In [12]:
#Reclassify masked slope raster. 0-3% = 1, 3-6% = 2, 6-12% = 3, 12-15% = 4, 15-18% = 8, 18-21% = 9, >21% = 10
slope_reclass = arcpy.sa.Reclassify(slope, 
                                    "VALUE", 
                                    "0 3 1;3 6 2;6 12 3;12 15 4;15 18 8;18 21 9;21 24 10;24 27 10;27 30 10;30 533.479797 10", 
                                    "DATA"); 
slope_reclass.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\slope_reclass")

#Land Cover: Extract raster by study extent and reclassify

In [13]:
#Use study extent as a mask for extracting NLCD raster
nlcd_study_extent = arcpy.sa.ExtractByMask(mn_nlcd, 
                                           study_extent, 
                                           "INSIDE", 
                                           '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); 
nlcd_study_extent.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\nlcd_se")

In [14]:
#Reclassify masked NLCD raster. Open Water = 10, Developed Open Space = 2, Developed Low Intensity = 2, Developed Medium Intensity = 2, Developed High Intensity = 2, Barren Land = 5, Deciduous Forest = 7, Evergreen Forest = 7, Mixed Forest = 7, Shrub = 7, Herbaceous = 7, Hay/Pasture = 9, Cultivated Crops = 9, Woody Wetlands = 9, Emergent Herbaceous Wetlands = 10
nlcd_reclass = arcpy.sa.Reclassify(nlcd_study_extent, 
                                   "NLCD_LAND", 
                                   "'Open Water' 10;'Developed, Open Space' 2;'Developed, Low Intensity' 2;'Developed, Medium Intensity' 2;'Developed, High Intensity' 2;'Barren Land' 5;'Deciduous Forest' 7;'Evergreen Forest' 7;'Mixed Forest' 7;Shrub/Scrub 7;Herbaceous 7;Hay/Pasture 9;'Cultivated Crops' 9;'Woody Wetlands' 9;'Emergent Herbaceous Wetlands' 10", 
                                   "DATA"); 
nlcd_reclass.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\nlcd_reclass")

#Streams: Clip feature class and reclassify

In [15]:
#Clip mn_streams shapefile within study extent
streams_clip = arcpy.analysis.Clip(mn_streams, 
                                   study_extent, 
                                   r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\streams_clip", 
                                   None)

In [16]:
#Convert streams shapefile to raster
streams_raster = arcpy.conversion.FeatureToRaster(streams_clip, 
                                                  "SO_VALUE", 
                                                  r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\stream_raster", 
                                                  r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\dem_se")

In [17]:
#Reclassify streams raster. Smaller streams are classified lower than larger streams.
streams_reclass = arcpy.sa.Reclassify(streams_raster, 
                                      "VALUE", 
                                      "1 1;2 2;3 8;4 8;5 10;6 10;8 10", 
                                      "NODATA"); 
streams_reclass.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\streams_rc")

In [19]:
#Performed Raster Calculator outside of script, because I continued to get errors, and filled in all non-stream areas with 0 value cells
final_streams_reclass = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\final_stream'

#Perform Weighted Overlay three different ways to generate three different Cost Surfaces

In [20]:
#All four input rasters have equal weights of 25%
cost_surface1 = arcpy.sa.WeightedOverlay(r"('C:\Users\cason\Desktop\GIS5571\Lab3\part1\slope_reclass' 25 'VALUE' (1 1; 2 2; 3 3; 4 4; 8 8; 9 9; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab3\part1\roads_reclass' 25 'VALUE' (1 1; 2 2; 3 3; 7 7; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab3\part1\nlcd_reclass' 25 'VALUE' (2 2; 5 5; 7 7; 9 9; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\final_stream' 25 'VALUE' (0 1; 1 1; 2 2; 8 8; 10 10; NODATA NODATA));1 10 1"); 
cost_surface1.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_surface1")

In [21]:
#All input rasters have weights of 30% except for roads, which has 10%
cost_surface2 = arcpy.sa.WeightedOverlay(r"('C:\Users\cason\Desktop\GIS5571\Lab3\part1\slope_reclass' 30 'VALUE' (1 1; 2 2; 3 3; 4 4; 8 8; 9 9; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab3\part1\roads_reclass' 10 'VALUE' (1 1; 2 2; 3 3; 7 7; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab3\part1\nlcd_reclass' 30 'VALUE' (2 2; 5 5; 7 7; 9 9; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\final_stream' 30 'VALUE' (0 1; 1 1; 2 2; 8 8; 10 10; NODATA NODATA));1 10 1"); 
cost_surface2.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_surface2")

In [25]:
#NLCD raster is weighted at 40%, streams raster is weighted at 30%, slope raster is weighted at 30%, and roads raster is not included
cost_surface3 = arcpy.sa.WeightedOverlay(r"('C:\Users\cason\Desktop\GIS5571\Lab3\part1\slope_reclass' 30 'VALUE' (1 1; 2 2; 3 3; 4 4; 8 8; 9 9; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab3\part1\nlcd_reclass' 40 'VALUE' (2 2; 5 5; 7 7; 9 9; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\final_stream' 30 'VALUE' (0 1; 1 1; 2 2; 8 8; 10 10; NODATA NODATA));1 10 1"); 
cost_surface3.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_surface3")

#Create variables for the location of Dory's farm and Dory's destination

In [23]:
#Create variables for CSV files created in Excel
start_csv = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\start.csv'
end_csv = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\end.csv'

In [24]:
#Create start point shapefile from CSV file
start = arcpy.management.XYTableToPoint(start_csv, 
                                        r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\start_point.shp", 
                                        "longitude", 
                                        "latitude", 
                                        None, 
                                        'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

In [26]:
#Create end point shapefile from CSV file
end = arcpy.management.XYTableToPoint(end_csv, 
                                      r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\end_point.shp", 
                                      "longitude", 
                                      "latitude", 
                                      None, 
                                      'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

#Perform First Cost Distance analysis

In [27]:
cost_distance1 = arcpy.sa.CostDistance(start, 
                                      cost_surface1, 
                                      None, 
                                      r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\direct_surf1", 
                                      None, None, None, None, ''); 
cost_distance1.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_dist_fin1")

RuntimeError: ERROR 010240: Could not save raster dataset to C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_dist_fin1 with output format GRID.

In [28]:
direction_surface1 = r'C:\Users\cason\Desktop\GIS5571\Lab3\part1\direct_surf1'

#Perform Second Cost Distance analysis

In [29]:
cost_distance2 = arcpy.sa.CostDistance(start, 
                                      cost_surface2, 
                                      None, 
                                      r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\direct_surf2", 
                                      None, None, None, None, ''); 
cost_distance2.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_dist_fin2")

RuntimeError: ERROR 010240: Could not save raster dataset to C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_dist_fin2 with output format GRID.

In [30]:
direction_surface2 = r'C:\Users\cason\Desktop\GIS5571\Lab3\part1\direct_surf2'

#Perform Third Cost Distance analysis

In [31]:
cost_distance3 = arcpy.sa.CostDistance(start, 
                                      cost_surface3, 
                                      None, 
                                      r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\direct_surf3", 
                                      None, None, None, None, ''); 
cost_distance3.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_dist_fin3")

RuntimeError: ERROR 010240: Could not save raster dataset to C:\Users\cason\Desktop\GIS5571\Lab3\part1\cost_dist_fin3 with output format GRID.

In [32]:
direction_surface3 = r'C:\Users\cason\Desktop\GIS5571\Lab3\part1\direct_surf3'

#Perform First Cost Path analysis

In [33]:
cost_path1 = arcpy.sa.CostPath(end, 
                               cost_distance1, 
                               direction_surface1, 
                               "BEST_SINGLE", 
                               "FID", 
                               "INPUT_RANGE"); 
cost_path1.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\optimal_route1")

RuntimeError: ERROR 010240: Could not save raster dataset to C:\Users\cason\Desktop\GIS5571\Lab3\part1\optimal_route1 with output format GRID.

#Perform Second Cost Path analysis

In [34]:
cost_path2 = arcpy.sa.CostPath(end, 
                               cost_distance2, 
                               direction_surface2, 
                               "BEST_SINGLE", 
                               "FID", 
                               "INPUT_RANGE"); 
cost_path2.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\optimal_route2")

RuntimeError: ERROR 010240: Could not save raster dataset to C:\Users\cason\Desktop\GIS5571\Lab3\part1\optimal_route2 with output format GRID.

#Perform Third Cost Path analysis

In [35]:
cost_path3 = arcpy.sa.CostPath(end, 
                               cost_distance3, 
                               direction_surface3, 
                               "BEST_SINGLE", 
                               "FID", 
                               "INPUT_RANGE"); 
cost_path3.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\part1\optimal_route3")

RuntimeError: ERROR 010240: Could not save raster dataset to C:\Users\cason\Desktop\GIS5571\Lab3\part1\optimal_route3 with output format GRID.